In [154]:
import requests
import json
import pandas as pd
import time
import re
import urllib
import matplotlib
import numpy as np
from datetime import datetime 
pd.set_option('display.max_colwidth', None)

In [159]:
def get_txs_after_date(address, last_date, api_endpoint):
    txs_df = None
    messages = []
    start = 0
    limit = 25
    while True:     
        url = api_endpoint.format(address, limit, start)
        print(url)
        time.sleep(5)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    
        response = requests.get(url, headers=headers)
        tt = pd.json_normalize(response.json(), record_path=['data', 'tx','body','messages'], meta=['data'])
        tt['txhash'] = tt.data.apply(lambda x: x['txhash'])
        tt['timestamp'] = pd.to_datetime(tt.data.apply(lambda x: x['timestamp']))
        tt = tt.drop(columns=['data'])
        tt['timestamp_str'] = tt.timestamp.dt.strftime("%Y-%m-%d %H:%M:%S")
        if(txs_df is None):
            txs_df = tt
        else:
            txs_df = pd.concat([txs_df, tt])
        messages.append(response.json())
        txs_before_date = tt[tt['timestamp_str'] < last_date]
        txs_from_date = tt[tt['timestamp_str'] >= last_date]
        if(len(txs_before_date) > 0):
            break
        start += limit
    return txs_from_date, messages

In [160]:
def get_atom_amount(amounts):
    for amount in amounts:
        if(amount['denom']=='uatom'):
            return int(amount['amount'])/1000000
            
def get_send_txs(df, addr, after_date):
    df = df[df.from_address==addr]
    df = df[df.timestamp_str >= after_date]
    df = df[df['@type']=='/cosmos.bank.v1beta1.MsgSend']
    df['atom_amount']= df.amount.apply(get_atom_amount)
    return df

In [161]:
def get_send_summary(df):
    dd = tt[['from_address','to_address','atom_amount','timestamp_str']]\
            .groupby(['from_address','to_address']).agg(tot_atom_amount=('atom_amount', np.sum),
                                                    first_send_timestamp=('timestamp_str', np.min))\
            .reset_index()
    return dd

In [162]:
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()
# Calculate one month ago by subtracting timedelta
one_month_ago = today - timedelta(days=90)
# Format the result as a string (optional)
formatted_date = one_month_ago.strftime('%Y-%m-%d %H:%M:%S')
print(formatted_date)

2023-05-01 08:26:20


In [163]:
txs_dict = {}
txs_polished_dict = {}
messages_dict = {}
send_chains_txs = None

In [164]:
addr = 'cosmos18e6wd0sscqlqprpl02lejln435y8aky8fme2jz'

In [165]:
_txs_df, _messages = get_txs_after_date(addr, formatted_date ,'https://api.mintscan.io/v1/cosmos/account/{}/txs?limit={}&from={}')

https://api.mintscan.io/v1/cosmos/account/cosmos18e6wd0sscqlqprpl02lejln435y8aky8fme2jz/txs?limit=25&from=0


In [166]:
txs_dict[addr] = _txs_df
messages_dict[addr] = _messages

In [167]:
txs_df = txs_dict[addr]
tt = txs_df[txs_df.txhash=='F98634D42DF053B52E80A0BCEB3CD00A1F20A6AA6772D2AABB3B782650868484']
tt = get_send_txs(tt, addr, formatted_date)
tt = get_send_summary(tt)
txs_polished_dict[addr] = tt
send_chains_txs = tt
send_chains_txs

,from_address,to_address,tot_atom_amount,first_send_timestamp
0,cosmos18e6wd0sscqlqprpl02lejln435y8aky8fme2jz,cosmos1p5tus3kyl8euzu385pwzcj00wu7rjkpqhzthpd,1.599191e+06,2023-05-29 15:17:41


In [168]:
txs_to_process = []
for i, row in send_chains_txs.iterrows():
    txs_to_process.append((row.from_address, row.to_address, row.first_send_timestamp))
txs_to_process

[('cosmos18e6wd0sscqlqprpl02lejln435y8aky8fme2jz',
  'cosmos1p5tus3kyl8euzu385pwzcj00wu7rjkpqhzthpd',
  '2023-05-29 15:17:41')]

In [170]:
i = 0
while True:
    #parse inputs
    row = txs_to_process[i]
    print(row)
    addr = row[1]
    date_ = row[2]
    #fetch data
    _txs_df, _messages = get_txs_after_date(addr, formatted_date ,'https://api.mintscan.io/v1/cosmos/account/{}/txs?limit={}&from={}')
    #store raw data
    txs_dict[addr] = _txs_df
    messages_dict[addr] = _messages
    #process data
    tt = txs_dict[addr]
    tt = get_send_txs(tt, addr, date_)
    tt = get_send_summary(tt)
    send_chains_txs = pd.concat([send_chains_txs,tt])
    #add to txs to process
    for i, row in tt.iterrows():
        txs_to_process.append((row.from_address, row.to_address, row.first_send_timestamp))
    print(txs_to_process)
    i += 1
    if(len(txs_to_process) <= i):
        break

('cosmos18e6wd0sscqlqprpl02lejln435y8aky8fme2jz', 'cosmos1p5tus3kyl8euzu385pwzcj00wu7rjkpqhzthpd', '2023-05-29 15:17:41')
https://api.mintscan.io/v1/cosmos/account/cosmos1p5tus3kyl8euzu385pwzcj00wu7rjkpqhzthpd/txs?limit=25&from=0


KeyError: "Key 'data' not found. To replace missing values of 'data' with np.nan, pass in errors='ignore'"

In [142]:
send_chains_txs.sort_values(by='first_send_timestamp')

,from_address,to_address,tot_atom_amount,first_send_timestamp
0,cosmos18e6wd0sscqlqprpl02lejln435y8aky8fme2jz,cosmos1p5tus3kyl8euzu385pwzcj00wu7rjkpqhzthpd,1.599191e+06,2023-05-29 15:17:41
0,cosmos1p5tus3kyl8euzu385pwzcj00wu7rjkpqhzthpd,cosmos1cms24y2e42d0q7wurxsk7ff8guu4lphd6jsza4,6.500500e+04,2023-05-31 02:54:41
1,cosmos1p5tus3kyl8euzu385pwzcj00wu7rjkpqhzthpd,cosmos1kfr242rvysart9k2ycz7l63t5yz9pvl7qlpyqt,1.275966e+06,2023-05-31 02:59:37
2,cosmos1p5tus3kyl8euzu385pwzcj00wu7rjkpqhzthpd,cosmos1w5mtmtuey5a3f59d9m0xh2quh90ze2vjj06rn2,6.100000e+05,2023-06-02 14:10:54


In [148]:
tt = [(1,'ciao')]
i = 0
while True:
    print(tt[i])
    if(tt[i][1]=='ciao'):
        tt.append((2,'tutto bene?'))
    if(tt[i][1]=='tutto bene?'):
        tt.append((3,'yes'))
    i += 1
    if(len(tt) <= i):
        break

(1, 'ciao')
(2, 'tutto bene?')
(3, 'yes')
